# Deforestation Dataset

This notebook prepares the deforestation dataset

In [ ]:
import os
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
import random
InteractiveShell.ast_node_interactivity = "all"
import os
from datetime import datetime, timedelta
from tqdm import tqdm
import cv2
import glob

from multiearth_challenge.datasets import segmentation_dataset as sd

from multiearth_challenge.datasets import base_datasets as bd

from multiearth_challenge import tiff_file_tools as tft

from dateutil.relativedelta import relativedelta

%matplotlib inline

In [ ]:
DATA_PATH = 'data/multiearth2023-dataset-final/'
DP_PATH = './dp'

# FIRE Target

In [ ]:
fire_target_files = glob.glob("data/multiearth2023-dataset-final/fire_train/Fire_ConfidenceLevel*.tiff", recursive=True)

In [ ]:
len(fire_target_files)

In [ ]:
tiff_path = fire_target_files[0]
tiff_path

In [ ]:
rows = []
for tiff_path in tqdm(fire_target_files):
    d = tft.parse_filename_parts(tiff_path)
    d['target_path'] = tiff_path
    rows.append(d)
fire_target = pd.DataFrame(rows)

In [ ]:
fire_target.shape
fire_target.head()

In [ ]:
fire_target.date = pd.to_datetime(fire_target.date)
fire_target['year'] = fire_target.date.dt.year
fire_target['month'] = fire_target.date.dt.month

fire_target = fire_target[fire_target.year >= 2013]
fire_target = fire_target[fire_target.year <= 2020]

fire_target = fire_target.sort_values(by=['year', 'month', 'lat', 'lon'])

In [ ]:
for _, row in tqdm(fire_target.iterrows()):
    img = cv2.imread(row['target_path'], -1)
    if img.max() > 0.5:
        1/0

In [ ]:
img.shape
img.max()

In [ ]:
fire_train = bd.NetCDFDataset(
    netcdf_file=DATA_PATH + 'fire_train.nc',
    data_filters=[],
    merge_bands=False,
)
len(fire_train)

# FOREST TARGET

In [ ]:
def mask2rle(x: np.ndarray) -> str:
    """
    Converts input masks into RLE-encoded strings.
    Source:
    https://www.kaggle.com/rakhlin/fast-run-length-encoding-python

    Args:
        x: numpy array of shape (height, width), 1 - mask, 0 - background
    Returns:
        RLE string
    """

    pixels = x.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return " ".join(str(x) for x in runs)

In [ ]:
rows = []
for mode in ["train", "val", "test"]:
    forest_target_files = glob.glob(f"data/multiearth2023-dataset-final/forest_{mode}/Defores*.tiff", recursive=True)
    print(mode, len(forest_target_files))

    for tiff_path in tqdm(forest_target_files):
        d = tft.parse_filename_parts(tiff_path)
        d['target_path'] = tiff_path
        d['mode'] = mode
        
        img = cv2.imread(tiff_path)
        img = img[:, :, 0]
        d['rle'] = mask2rle(img)
        d['img_mean'] = img.mean()
        rows.append(d)
forest_target = pd.DataFrame(rows)

In [ ]:
roi = {(r.lat, r.lon, r.date.strftime("%Y-%m-%d")) for _, r in forest_target.iterrows()}
len(roi)

In [ ]:
d['date']

In [ ]:
forest_target.groupby('mode').img_mean.mean()

In [ ]:
forest_target.to_csv('forest_target.csv', index=False)

In [ ]:
np.mean(forest_target['img_mean'] > 0)

In [ ]:
forest_target.shape
forest_target.head()
forest_target.tail()

In [ ]:
satellite_datasets = [
    
]
ds_name = 'landsat8_train.nc'

In [ ]:
ds = bd.NetCDFDataset(
    netcdf_file=DATA_PATH + ds_name,
    data_filters=[],
    merge_bands=False,
)
len(ds)

In [ ]:
for i, d in tqdm(enumerate(ds)):
    
    band = d['bands'][0]
    lat, lon = d['lat_lon'].round(2)
    
    
    source_date = pd.to_datetime(d['date'])

    m0 = datetime(source_date.year, source_date.month, 1)
    m1 = m0 + relativedelta(months=1)
    m2 = m0 + relativedelta(months=2)
    
    # Search all source images in 3 months 
    if ((lat, lon, m0.strftime("%Y-%m-%d")) in roi) or \
        ((lat, lon, m1.strftime("%Y-%m-%d")) in roi) or \
            ((lat, lon, m2.strftime("%Y-%m-%d")) in roi):
        img = d['image'][0]
        os.makedirs(f"{SOURCE_DIR}/{band}/{source_year}/", exist_ok=True)
        source_date = source_date.strftime("%Y-%m-%d")
        source_year = source_date[:4]
        source_key = f"{source}_{round(lat, 2)}_{round(lon, 2)}_{source_date}"
    if i > 1000:
        1/0
        

In [ ]:
d

In [ ]:
img

source_date, month_start, next_month

In [ ]:
plt.imshow()
band§a

In [ ]:
TARGET = 'deforestation'
band = 'deforestation'
TARGET_DIR = f"{DP_PATH}/{TARGET}"
os.makedirs(TARGET_DIR, exist_ok=True)

In [ ]:
rows = []
for target_data in tqdm(forest_train):
    lat, lon = target_data['lat_lon']
    target_date = target_data['date']
    target_date_str = pd.to_datetime(target_date).strftime("%Y-%m-%d")

    target_key = f"{TARGET}_{round(lat, 2)}_{round(lon, 2)}_{target_date_str}"
    os.makedirs(f"{TARGET_DIR}/{band}", exist_ok=True)
    target_path = f"{TARGET_DIR}/{band}/{target_key}.npy"
    
    img = target_data['image'][0]
    np.save(target_path, img)
    
    target_mean = img.mean()
    
    rle = mask2rle(img)
    
    row = [target_path, band, lat, lon, target_date, rle, target_mean]
    rows.append(row)

In [ ]:
lat, lon = target_data['lat_lon']
target_date = target_data['date']
target_date_str = pd.to_datetime(target_date).strftime("%Y-%m-%d")

In [ ]:
pd.DataFrame(rows, columns=['target_path', 'band', 'lat', 'lon', 'target_date', 'rle', 'target_mean'])

In [ ]:
img

In [ ]:
img.mean()

In [ ]:
mask2rle(img)

In [ ]:
fs = glob.glob("dp/**/*.npy", recursive=True)
len(fs)

In [ ]:
for f in fs:

In [ ]:
!ls

In [ ]:
ls dp

In [ ]:
metas = [
    'sat_ls8_landsat8_train_meta.csv',
    'sat_s2_sent2_b5-b8_train_meta.csv',
    'sat_s1_sent1_train_meta.csv',
    'sat_s2_sent2_b9-b12_train_meta.csv',
    'sat_s2_sent2_b1-b4_train_meta.csv',
]
tile_stats = pd.concat([pd.read_csv(f"./dp/{f}") for f in metas])

In [ ]:
pd.concat([
    tile_stats.groupby(['source', 'band']).imin.min(),
    tile_stats.groupby(['source', 'band']).imin.mean(),
    tile_stats.groupby(['source', 'band']).imean.mean(),
    tile_stats.groupby(['source', 'band']).imax.mean(),
    tile_stats.groupby(['source', 'band']).imax.max(),

], axis=1)

In [ ]:
tile_stats.head()

In [ ]:
new_rows = []
for _, row in tqdm(tile_stats.sample(frac=0.5).iterrows()):
    x = np.load(row.source_path)
    d = {
        "xmin": x.min(),
        "p5": np.percentile(x, 5),
        "p50": np.percentile(x, 50),
        "xmean": x.mean(),
        "p95": np.percentile(x, 95),
        "xmax": x.max(),
        "xstd": x.std(),
    }
    new_row = row.to_dict()
    new_row.update(d)
    new_rows.append(new_row)

In [ ]:
tile_stats.groupby(['source', 'lat', 'lon', 'source_date']).count()

In [ ]:
new_stats = pd.DataFrame(new_rows)

In [ ]:
new_stats.groupby(['source', 'band'])[[
    'xmin', 'p5', 'p50', 'xmean', 'p95', 'xmax', 'xstd'
]].mean().reset_index()

In [ ]:
new_stats.groupby(['source', 'band'])[[
    'xmin', 'p5', 'p50', 'xmean', 'p95', 'xmax', 'xstd'
]].quantile(0.9).reset_index()

In [ ]:
new_stats.groupby(['source', 'band']).p99.max()

In [ ]:
forest_target.groupby(['lat', 'lon']).nunique()